In [1]:
from utils.preprocessing import DataLoader
import gurobipy as gp
from gurobipy import *
import math

# 数据读取

In [2]:
data_loader = DataLoader('data/Data A-Crew.csv', 'data/Data A-Flight.csv')

In [3]:
data_loader.dump_data()

Counting times...(0/206)
Counting times...(100/206)
Counting times...(200/206)
## Loading C
## C loaded
## Loading AP
## AP loaded
## Loading Base
## Base loaded
## Loading F
## F loaded
## Loading FF
Loading data... (10000/42436)
Loading data... (20000/42436)
Loading data... (30000/42436)
Loading data... (40000/42436)
## FF loaded
## Loading FF1
Loading data... (10000/42436)
Loading data... (20000/42436)
Loading data... (30000/42436)
Loading data... (40000/42436)
## FF1 loaded
## Loading FF2
Loading data... (10000/42436)
Loading data... (20000/42436)
Loading data... (30000/42436)
Loading data... (40000/42436)
## FF2 loaded
Counting times...(0/206)
Counting times...(100/206)
Counting times...(200/206)


In [4]:
data_loader_a = DataLoader('data/Data A-Crew.csv', 'data/Data A-Flight.csv')

In [5]:
data_loader_a.dump_data(cropped_date=(11, 15))

Counting times...(0/64)
## Loading C
## C loaded
## Loading AP
## AP loaded
## Loading Base
## Base loaded
## Loading F
## F loaded
## Loading FF
## FF loaded
## Loading FF1
## FF1 loaded
## Loading FF2
## FF2 loaded
Counting times...(0/64)


# 模型建立与求解

## 第一阶段

### 优化目标①

In [6]:
MaxTAFB = 14400 / 15 * 5

m=gp.Model('m1')

z=m.addVars(data_loader_a.F,vtype=gp.GRB.BINARY,name='z')
x_ikdh=m.addVars(data_loader_a.F,data_loader_a.C,vtype=gp.GRB.BINARY,name='x_dh')
x_ikfo=m.addVars(data_loader_a.F,data_loader_a.C,vtype=gp.GRB.BINARY,name='x_fo')
x_ikcap=m.addVars(data_loader_a.F,data_loader_a.C,vtype=gp.GRB.BINARY,name='x_cap')

r_iksta=m.addVars(data_loader_a.F,data_loader_a.C,vtype=gp.GRB.BINARY,name='r_iksta')
r_jkfin=m.addVars(data_loader_a.F,data_loader_a.C,vtype=gp.GRB.BINARY,name='r_jkfin')
d_iksta=m.addVars(data_loader_a.F,data_loader_a.C,vtype=gp.GRB.BINARY,name='d_iksta')
d_jkfin=m.addVars(data_loader_a.F,data_loader_a.C,vtype=gp.GRB.BINARY,name='d_jkfin')
p_iksta=m.addVars(data_loader_a.F,data_loader_a.C,vtype=gp.GRB.BINARY,name='p_iksta')
p_jkfin=m.addVars(data_loader_a.F,data_loader_a.C,vtype=gp.GRB.BINARY,name='p_jkfin')

y_ijk=m.addVars(data_loader_a.F,data_loader_a.F,data_loader_a.C,vtype=gp.GRB.BINARY,name='y_ijk')
v_ijk=m.addVars(data_loader_a.F,data_loader_a.F,data_loader_a.C,vtype=gp.GRB.BINARY,name='v_ijk')
w_ijk=m.addVars(data_loader_a.F,data_loader_a.F,data_loader_a.C,vtype=gp.GRB.BINARY,name='w_ijk')
u_ijk=m.addVars(data_loader_a.F,data_loader_a.F,data_loader_a.C,vtype=gp.GRB.BINARY,name='u_ijk')

# a1=m.addVar(1,vtype=gp.GRB.INTEGER,name='a1')
# a2=m.addVar(2,vtype=gp.GRB.INTEGER,name='a2')

# a3=m.addVar(3,vtype=gp.GRB.INTEGER,name='a3')
# a4=m.addVar(4,vtype=gp.GRB.INTEGER,name='a4')


m.ModelSense=GRB.MINIMIZE

m.setObjective(-z.sum())


# m.setObjective(gp.quicksum(gp.quicksum(data_loader_a.DCost[k]*(gp.quicksum(d_jkfin[i,k]*data_loader_a.arrivetime[i] for i in data_loader_a.FD[t]) 
#                                                                - gp.quicksum(d_iksta[i,k]*data_loader_a.leavetime[i] for i in data_loader_a.FD[t]))/60
#                                                            for k in data_loader_a.C) for t in data_loader_a.Dates ))
# m.setObjective(gp.quicksum(gp.quicksum(data_loader_a.PCost[k]*(u_ijk[i,j,k]*data_loader_a.arrivetime[j] - u_ijk[i,j,k]*data_loader_a.leavetime[i]) for i in data_loader_a.FD[t] for j in data_loader_a.FD[t])/60
#                                                            for k in data_loader_a.C for t in data_loader_a.Dates ))
# m.setObjectiveN(x_ikdh.sum(), index=2, priority=8)


# m.setObjectiveN(a1-a2, index=3, priority=7)
# m.setObjectiveN(a3-a4, index=4, priority=6)
# m.setObjectiveN(gp.quicksum(x_ikfo[i,k] for i in data_loader_a.F for k in data_loader_a.C2), index=5, priority=5)

# m.setObjectiveN(-z.sum(),index = 0,weight =0.8)
# m.setObjectiveN(x_ikdh.sum(),index=1,weight=0.2)
# m.setObjectiveN(gp.quicksum(x_ikfo[i,k] for i in data_loader_a.F for k in data_loader_a.C2),index=2,weight=0.05)


M=10000
#对X的约束
m.addConstrs(x_ikfo[i,k]==0 for i in data_loader_a.F for k in data_loader_a.C1 )
m.addConstrs(x_ikcap[i,k]==0 for i in data_loader_a.F for k in data_loader_a.C3 )
m.addConstrs(x_ikcap[i,k]+x_ikfo[i,k]+x_ikdh[i,k]<=1 for i in data_loader_a.F for k in data_loader_a.C)

#对Z的约束
m.addConstrs(gp.quicksum(x_ikcap[i,k]+x_ikfo[i,k]+x_ikdh[i,k] for k in data_loader_a.C)<=M*(z[i]) for i in data_loader_a.F)
m.addConstrs(x_ikcap.sum(i,'*')== z[i] for i in data_loader_a.F )
m.addConstrs(x_ikfo.sum(i,'*')== z[i] for i in data_loader_a.F )
m.addConstrs(M*z[j]>=gp.quicksum(x_ikcap[j,k]+x_ikfo[j,k]+x_ikdh[j,k] for k in data_loader_a.C) for j in data_loader_a.F)

#对Y的约束
m.addConstrs(y_ijk[i,j,k]==0 for i,j in data_loader_a.FF for k in data_loader_a.C)
#m.addConstrs(gp.quicksum(y_ijk[i,j,k] for j in F)<=x_ikcap[i,k]+x_ikfo[i,k]+x_ikdh[i,k] for i in F for k in C)
#m.addConstrs(gp.quicksum(y_ijk[j,i,k] for j in F)<=x_ikcap[i,k]+x_ikfo[i,k]+x_ikdh[i,k] for i in F for k in C)

#对roaster周期的约束
m.addConstrs(r_jkfin[j,k]== 0 for j in data_loader_a.nonF_arrive_base[0] for k in data_loader_a.C)
m.addConstrs(r_iksta[i,k]== 0 for i in data_loader_a.nonF_leave_base[0] for k in data_loader_a.C)

m.addConstrs(gp.quicksum(r_iksta[i,k] for i in data_loader_a.F_leave_base[0] )<=1  for k in data_loader_a.C)
m.addConstrs(gp.quicksum(r_jkfin[j,k] for j in data_loader_a.F_arrive_base[0] )-gp.quicksum(r_iksta[i,k] for i in data_loader_a.F_leave_base[0] ) == 0 for k in data_loader_a.C)

#第一问中航班对应周期的约束
m.addConstrs(1-(y_ijk.sum(i,'*',k)+r_iksta[i,k]+r_jkfin[i,k])<=M*(1-(x_ikcap[i,k]+x_ikfo[i,k]+x_ikdh[i,k])) for i in data_loader_a.F for k in data_loader_a.C)
m.addConstrs(y_ijk.sum(i,'*',k)+r_jkfin[i,k]-(y_ijk.sum('*',i,k)+r_iksta[i,k]) == 0 for i in data_loader_a.F for k in data_loader_a.C)

m.addConstrs(y_ijk.sum(i,'*',k)+r_jkfin[i,k] <= x_ikcap[i,k]+x_ikfo[i,k]+x_ikdh[i,k]  for i in data_loader_a.F for k in data_loader_a.C)
m.addConstrs(y_ijk.sum('*',i,k)+r_iksta[i,k] <= x_ikcap[i,k]+x_ikfo[i,k]+x_ikdh[i,k]  for i in data_loader_a.F for k in data_loader_a.C)

#第二问
#对V的约束
m.addConstrs(v_ijk[i,j,k]==0 for i,j in data_loader_a.FF1 for k in data_loader_a.C)
m.addConstrs(v_ijk[i,j,k] <= y_ijk[i,j,k] for i in data_loader_a.F for j in data_loader_a.F for k in data_loader_a.C)


#对duty执勤的约束
m.addConstrs(gp.quicksum(d_iksta[i,k] for i in data_loader_a.FD[t]) - gp.quicksum(d_jkfin[i,k] for i in data_loader_a.FD[t]) == 0 for i in data_loader_a.F for k in data_loader_a.C for t in data_loader_a.Dates)
m.addConstrs(gp.quicksum(d_iksta[i,k] for i in data_loader_a.FD[t])   <= gp.quicksum(x_ikcap[i,k]+x_ikfo[i,k]+x_ikdh[i,k] for i in data_loader_a.FD[t]) for i in data_loader_a.F for k in data_loader_a.C for t in data_loader_a.Dates)
m.addConstrs(gp.quicksum(d_iksta[i,k] for i in data_loader_a.FD[t])*M >= gp.quicksum(x_ikcap[i,k]+x_ikfo[i,k]+x_ikdh[i,k] for i in data_loader_a.FD[t]) for k in data_loader_a.C for t in data_loader_a.Dates)

#m.addConstrs(d_iksta[i,k] for i in data_loader_a.FD[t]  <= x_ikcap[i,k]+x_ikfo[i,k]+x_ikdh[i,k] for i in data_loader_a.FD[t] for k in data_loader_a.C for t in data_loader_a.Dates)

#第二问中航班对应执勤，执勤对应周期的约束
m.addConstrs((v_ijk.sum(i,'*',k)+ r_jkfin[i,k] ==  d_jkfin[i,k]) for i in data_loader_a.F for k in data_loader_a.C)
m.addConstrs((v_ijk.sum('*',i,k)+ r_iksta[i,k] ==  d_iksta[i,k]) for i in data_loader_a.F for k in data_loader_a.C)

#m.addConstrs((d_jkfin[i,k] <= x_ikcap[i,k]+x_ikfo[i,k]+x_ikdh[i,k])  for i in data_loader_a.F for k in data_loader_a.C)
#m.addConstrs((d_iksta[i,k] <= x_ikcap[i,k]+x_ikfo[i,k]+x_ikdh[i,k])  for i in data_loader_a.F for k in data_loader_a.C)

#第二问中其余约束
m.addConstrs(gp.quicksum(d_jkfin[i,k]*data_loader_a.arrivetime[i] for i in data_loader_a.FD[t]) - gp.quicksum(d_iksta[i,k]*data_loader_a.leavetime[i] for i in data_loader_a.FD[t]) <= 720 for k in data_loader_a.C for t in data_loader_a.Dates )
m.addConstrs(gp.quicksum((x_ikcap[i,k]+x_ikfo[i,k])*(data_loader_a.leavetime[i]-data_loader_a.arrivetime[i]) for i in data_loader_a.FD[t]) <= 600 for k in data_loader_a.C for t in data_loader_a.Dates )
#m.addConstrs(gp.quicksum(d_jkfin[i,k]*data_loader_a.arrivetime[i] for i in data_loader_a.FD[t]) - gp.quicksum(d_iksta[i,k]*data_loader_a.leavetime[i] for i in data_loader_a.FD[t]) >= 0 for k in data_loader_a.C for t in data_loader_a.Dates )


#m.addConstr(z.sum('*')==206)
# m.addConstr(gp.quicksum(gp.quicksum(data_loader_a.DCost[k]*(gp.quicksum(d_jkfin[i,k]*data_loader_a.arrivetime[i] for i in data_loader_a.FD[t]) 
#                     - gp.quicksum(d_iksta[i,k]*data_loader_a.leavetime[i] for i in data_loader_a.FD[t]))
#                 for k in data_loader_a.C) for t in data_loader_a.Dates )==[1324573-100000,1324573+100000])
#对执勤时长平衡的约束（辅助目标）
# m.addConstrs(a1>=(gp.quicksum(gp.quicksum(d_jkfin[i,k]*data_loader_a.arrivetime[i] for i in data_loader_a.FD[t]) for t in data_loader_a.Dates) - gp.quicksum(gp.quicksum(d_iksta[i,k]*data_loader_a.leavetime[i] for i in data_loader_a.FD[t]) for t in data_loader_a.Dates)) for k in data_loader_a.C)
# m.addConstrs(a2<=(gp.quicksum(gp.quicksum(d_jkfin[i,k]*data_loader_a.arrivetime[i] for i in data_loader_a.FD[t]) for t in data_loader_a.Dates) - gp.quicksum(gp.quicksum(d_iksta[i,k]*data_loader_a.leavetime[i] for i in data_loader_a.FD[t]) for t in data_loader_a.Dates)) for k in data_loader_a.C)
         
#第三问
#对W的约束
m.addConstrs(w_ijk[i,j,k]==0 for i,j in data_loader_a.FF2 for k in data_loader_a.C)
m.addConstrs(w_ijk[i,j,k] <= v_ijk[i,j,k] for i in data_loader_a.F for j in data_loader_a.F for k in data_loader_a.C)



#对U的约束
m.addConstrs(r_jkfin[i,k] <=gp.quicksum(u_ijk[j,i,k] for j in data_loader_a.F) for i in data_loader_a.F for k in data_loader_a.C)
m.addConstrs(r_iksta[i,k] <=gp.quicksum(u_ijk[i,j,k] for j in data_loader_a.F) for i in data_loader_a.F for k in data_loader_a.C)

m.addConstrs(gp.quicksum(u_ijk[j,i,k] for j in data_loader_a.F) <= d_jkfin[i,k] for i in data_loader_a.F for k in data_loader_a.C)
m.addConstrs(gp.quicksum(u_ijk[i,j,k] for j in data_loader_a.F) <= d_iksta[i,k] for i in data_loader_a.F for k in data_loader_a.C)

m.addConstrs(gp.quicksum(u_ijk[i,j,k] for j in data_loader_a.F)-(r_iksta[i,k] + gp.quicksum(w_ijk[j,i,k] for j in data_loader_a.F)) == 0 for i in data_loader_a.F for k in data_loader_a.C)
m.addConstrs(gp.quicksum(u_ijk[j,i,k] for j in data_loader_a.F)-(r_jkfin[i,k] + gp.quicksum(w_ijk[i,j,k] for j in data_loader_a.F))== 0 for i in data_loader_a.F for k in data_loader_a.C)


m.addConstrs(u_ijk[i,j,k]*(data_loader_a.leavedate[j] - data_loader_a.leavedate[i] )<= 4 for i in data_loader_a.F for j in data_loader_a.F for k in data_loader_a.C)
#m.addConstrs(w_ijk[i,j,k]*(data_loader_a.leavedate[j] - data_loader_a.leavedate[i] )>= 2 for i in data_loader_a.F for j in data_loader_a.F for k in data_loader_a.C)
m.addConstrs(gp.quicksum(u_ijk[i,j,k]*(data_loader_a.arrivetime[j] - data_loader_a.leavetime[i])for i in data_loader_a.F for j in data_loader_a.F )<= MaxTAFB for k in data_loader_a.C)       
m.addConstrs(u_ijk[i,j,k]==0 for i,j in data_loader_a.FF for k in data_loader_a.C)

#对执勤时长平衡的约束（辅助目标）
# m.addConstrs(a3>=(gp.quicksum(gp.quicksum(v_ijk[i,j,k]*data_loader_a.arrivetime[j] - v_ijk[i,j,k]*data_loader_a.leavetime[i] for i in data_loader_a.FD[t] for j in data_loader_a.FD[t]) for t in data_loader_a.Dates )) for k in data_loader_a.C)
# m.addConstrs(a4<=(gp.quicksum(gp.quicksum(v_ijk[i,j,k]*data_loader_a.arrivetime[j] - v_ijk[i,j,k]*data_loader_a.leavetime[i] for i in data_loader_a.FD[t] for j in data_loader_a.FD[t]) for t in data_loader_a.Dates )) for k in data_loader_a.C)

# m.addConstr(z.sum()==64)
# m.addConstr(gp.quicksum(gp.quicksum(data_loader_a.DCost[k]*(gp.quicksum(d_jkfin[i,k]*data_loader_a.arrivetime[i] for i in data_loader_a.FD[t]) 
#                                                           - gp.quicksum(d_iksta[i,k]*data_loader_a.leavetime[i] for i in data_loader_a.FD[t]))/60
#                                                           for k in data_loader_a.C) for t in data_loader_a.Dates) == 175316)

m.update()
m.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-02
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 599548 rows, 356224 columns and 2673445 nonzeros
Model fingerprint: 0x6d2456e7
Variable types: 0 continuous, 356224 integer (356224 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Found heuristic solution: objective 0.0000000
Presolve removed 583294 rows and 329634 columns (presolve time = 5s) ...
Presolve removed 583294 rows and 329634 columns
Presolve time: 6.06s
Presolved: 16254 rows, 26590 columns, 119348 nonzeros
Variable types: 0 continuous, 26590 integer (26590 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.

In [7]:
for k in data_loader_a.C:
    sum1 = 0
    for i in data_loader_a.F:
        for j in data_loader_a.F:
            if u_ijk[i,j,k].x > 0.9:
                sum1 += data_loader_a.arrivetime[j] - data_loader_a.leavetime[i]
    print(str(k)+": "+str(sum1))

0: 4640
1: 4540
2: 3140
3: 430
4: 4580
5: 1950
6: 4640
7: 2010
8: 4580
9: 3200
10: 4565
11: 220
12: 3200
13: 475
14: 1760
15: 1660
16: 0
17: 4355
18: 4565
19: 4575
20: 3140


In [8]:
data_loader_a.PCost

{0: 20,
 1: 20,
 2: 20,
 3: 20,
 4: 20,
 5: 20,
 6: 20,
 7: 20,
 8: 20,
 9: 20,
 10: 20,
 11: 20,
 12: 20,
 13: 20,
 14: 20,
 15: 20,
 16: 20,
 17: 20,
 18: 20,
 19: 20,
 20: 20}

In [9]:
loop_dic = {}
for i in data_loader_a.F:
    for j in data_loader_a.F:
        for k in data_loader_a.C:
            if u_ijk[i,j,k].x>0.9:
                if data_loader_a.C_dic[k] in loop_dic.keys():
                    loop_dic[data_loader_a.C_dic[k]].append(data_loader_a.leavedate[j]-data_loader_a.leavedate[i]+1)
                else:
                    loop_dic[data_loader_a.C_dic[k]]=[data_loader_a.leavedate[j]-data_loader_a.leavedate[i]+1]
loop_dic

{'A0016': [2],
 'A0002': [4],
 'A0012': [1],
 'A0014': [1, 1],
 'A0005': [4],
 'A0009': [4],
 'A0003': [3],
 'A0021': [3],
 'A0008': [1, 2],
 'A0018': [4],
 'A0011': [4],
 'A0019': [4],
 'A0006': [1, 2],
 'A0004': [1, 1],
 'A0020': [4],
 'A0015': [2],
 'A0010': [3],
 'A0013': [3],
 'A0001': [4],
 'A0007': [4]}

In [10]:
sum = 0
for i in data_loader_a.F:
    for j in data_loader_a.F:
        for k in data_loader_a.C:
            sum+=data_loader_a.PCost[k]*(u_ijk[i,j,k].x*data_loader_a.arrivetime[j] - u_ijk[i,j,k].x*data_loader_a.leavetime[i])/60
sum

20741.666666666668

In [11]:
results_dic = {'em_no':[], 'fl_no':[], 'cls':[]}

In [12]:
for i in data_loader_a.F:
    for k in data_loader_a.C:
        if x_ikcap[i,k].x > 0.9:
            results_dic['em_no'].append(k)
            for key, value in data_loader.F_dic.items():
                if value == data_loader_a.F_dic[i]:
                    results_dic['fl_no'].append(key)
                    break
            results_dic['cls'].append(1)
        if x_ikfo[i,k].x > 0.9:
            results_dic['em_no'].append(k)
            for key, value in data_loader.F_dic.items():
                if value == data_loader_a.F_dic[i]:
                    results_dic['fl_no'].append(key)
                    break
            results_dic['cls'].append(2)
        if x_ikdh[i,k].x > 0.9:
            results_dic['em_no'].append(k)
            for key, value in data_loader.F_dic.items():
                if value == data_loader_a.F_dic[i]:
                    results_dic['fl_no'].append(key)
                    break
            results_dic['cls'].append(3)

In [13]:
for i in data_loader_a.F:
    for k in data_loader_a.C:
        if r_iksta[i,k].x > 0.9:
            print(str(data_loader_a.C_dic[k])+"出发于"+str(data_loader_a.AP_dic[data_loader_a.F_ap_dpt_dic[i]])+"乘坐"+str(data_loader_a.F_dic[i])+"到达"+str(data_loader_a.AP_dic[data_loader_a.F_ap_arr_dic[i]]))

A0002出发于NKX乘坐FA680/11到达PGX
A0016出发于NKX乘坐FA680/11到达PGX
A0012出发于NKX乘坐FA680/12到达PGX
A0005出发于NKX乘坐FA812/11到达PDK
A0009出发于NKX乘坐FA812/11到达PDK
A0003出发于NKX乘坐FA812/13到达PDK
A0021出发于NKX乘坐FA812/13到达PDK
A0008出发于NKX乘坐FA854/11到达CTH
A0018出发于NKX乘坐FA854/11到达CTH
A0011出发于NKX乘坐FA854/12到达CTH
A0019出发于NKX乘坐FA854/12到达CTH
A0006出发于NKX乘坐FA864/11到达PXB
A0014出发于NKX乘坐FA864/11到达PXB
A0004出发于NKX乘坐FA864/12到达PXB
A0020出发于NKX乘坐FA864/12到达PXB
A0001出发于NKX乘坐FA884/11到达XGS
A0007出发于NKX乘坐FA884/11到达XGS
A0010出发于NKX乘坐FA884/11到达XGS
A0013出发于NKX乘坐FA884/11到达XGS
A0015出发于NKX乘坐FA884/11到达XGS


In [14]:
temp_leave = {}
for i in data_loader_a.F:
    for k in data_loader_a.C:
        if r_jkfin[i,k].x > 0.9:
            print(str(data_loader_a.C_dic[k])+"乘坐"+str(data_loader_a.F_dic[i]))
            temp_leave[k] = i
temp_leave

A0016乘坐FA3/12
A0012乘坐FA681/12
A0002乘坐FA681/14
A0014乘坐FA681/14
A0005乘坐FA813/14
A0009乘坐FA813/14
A0003乘坐FA813/15
A0021乘坐FA813/15
A0018乘坐FA855/14
A0011乘坐FA855/15
A0019乘坐FA855/15
A0006乘坐FA865/15
A0020乘坐FA865/15
A0004乘坐FA873/15
A0015乘坐FA885/12
A0010乘坐FA885/13
A0013乘坐FA885/13
A0001乘坐FA885/14
A0007乘坐FA885/14
A0008乘坐FA885/15


{15: 1,
 11: 8,
 1: 10,
 13: 10,
 4: 20,
 8: 20,
 2: 21,
 20: 21,
 17: 30,
 10: 31,
 18: 31,
 5: 41,
 19: 41,
 3: 49,
 14: 56,
 9: 57,
 12: 57,
 0: 58,
 6: 58,
 7: 59}

In [15]:
data_loader_a2 = DataLoader('data/Data A-Crew.csv', 'data/Data A-Flight.csv')
data_loader_a2.dump_data(cropped_date=(16, 20))

Counting times...(0/72)
## Loading C
## C loaded
## Loading AP
## AP loaded
## Loading Base
## Base loaded
## Loading F
## F loaded
## Loading FF
## FF loaded
## Loading FF1
## FF1 loaded
## Loading FF2
## FF2 loaded
Counting times...(0/72)


In [16]:
data_loader_a2.F_dic

{0: 'FA680/16',
 1: 'FA680/17',
 2: 'FA680/18',
 3: 'FA680/19',
 4: 'FA680/20',
 5: 'FA681/16',
 6: 'FA681/17',
 7: 'FA681/18',
 8: 'FA681/19',
 9: 'FA681/20',
 10: 'FA812/16',
 11: 'FA812/17',
 12: 'FA812/18',
 13: 'FA812/19',
 14: 'FA812/20',
 15: 'FA813/16',
 16: 'FA813/17',
 17: 'FA813/18',
 18: 'FA813/19',
 19: 'FA813/20',
 20: 'FA854/16',
 21: 'FA854/17',
 22: 'FA854/18',
 23: 'FA854/19',
 24: 'FA854/20',
 25: 'FA855/16',
 26: 'FA855/17',
 27: 'FA855/18',
 28: 'FA855/19',
 29: 'FA855/20',
 30: 'FA864/16',
 31: 'FA864/17',
 32: 'FA864/18',
 33: 'FA864/19',
 34: 'FA864/20',
 35: 'FA865/16',
 36: 'FA865/17',
 37: 'FA865/18',
 38: 'FA865/19',
 39: 'FA865/20',
 40: 'FA872/16',
 41: 'FA872/17',
 42: 'FA872/18',
 43: 'FA872/19',
 44: 'FA872/20',
 45: 'FA873/16',
 46: 'FA873/17',
 47: 'FA873/18',
 48: 'FA873/19',
 49: 'FA873/20',
 50: 'FA884/16',
 51: 'FA884/17',
 52: 'FA884/18',
 53: 'FA884/19',
 54: 'FA884/20',
 55: 'FA885/16',
 56: 'FA885/17',
 57: 'FA885/18',
 58: 'FA885/19',
 59: 'F

In [17]:
for k in data_loader_a2.C:
    for key, value in data_loader_a2.F_dic.items():
        if value == temp_leave[k]:
            temp_leave[k] = key
            break

KeyError: 16

In [ ]:
temp_leave

In [ ]:
for key, value in temp_leave.items():
    print(str(data_loader_a2.C_dic[key])+"乘坐"+str(data_loader_a2.F_dic[value])+"到达"+str(data_loader_a2.AP_dic[data_loader_a2.F_ap_arr_dic[value]]))

In [ ]:
data_loader_a2.leavedate

In [ ]:
temp_leave_dic = {}
MinRest = 2
for k in data_loader_a2.C:
    for i in data_loader_a2.F:
        if k not in temp_leave_dic.keys():
            temp_leave_dic[k] = []
        if (data_loader_a2.AP_dic[data_loader_a2.F_ap_dpt_dic[i]] == 'NKX') and ((data_loader_a2.leavedate[i] - data_loader_a.leavedate[temp_leave[k]])>MinRest):#不是到达的机场
            continue
        else:
            if k not in temp_leave_dic.keys(): 
                temp_leave_dic[k] = [i]
            else:
                temp_leave_dic[k].append(i)

In [ ]:
temp_leave_dic

In [ ]:
len(temp_leave_dic[3])

## 第二阶段

### 优化目标①

In [ ]:
MaxTAFB = 14400 / 15 * 5

m=gp.Model('m1')

z=m.addVars(data_loader_a2.F,vtype=gp.GRB.BINARY,name='z')
x_ikdh=m.addVars(data_loader_a2.F,data_loader_a2.C,vtype=gp.GRB.BINARY,name='x_dh')
x_ikfo=m.addVars(data_loader_a2.F,data_loader_a2.C,vtype=gp.GRB.BINARY,name='x_fo')
x_ikcap=m.addVars(data_loader_a2.F,data_loader_a2.C,vtype=gp.GRB.BINARY,name='x_cap')

r_iksta=m.addVars(data_loader_a2.F,data_loader_a2.C,vtype=gp.GRB.BINARY,name='r_iksta')
r_jkfin=m.addVars(data_loader_a2.F,data_loader_a2.C,vtype=gp.GRB.BINARY,name='r_jkfin')
d_iksta=m.addVars(data_loader_a2.F,data_loader_a2.C,vtype=gp.GRB.BINARY,name='d_iksta')
d_jkfin=m.addVars(data_loader_a2.F,data_loader_a2.C,vtype=gp.GRB.BINARY,name='d_jkfin')
p_iksta=m.addVars(data_loader_a2.F,data_loader_a2.C,vtype=gp.GRB.BINARY,name='p_iksta')
p_jkfin=m.addVars(data_loader_a2.F,data_loader_a2.C,vtype=gp.GRB.BINARY,name='p_jkfin')

y_ijk=m.addVars(data_loader_a2.F,data_loader_a2.F,data_loader_a2.C,vtype=gp.GRB.BINARY,name='y_ijk')
v_ijk=m.addVars(data_loader_a2.F,data_loader_a2.F,data_loader_a2.C,vtype=gp.GRB.BINARY,name='v_ijk')
w_ijk=m.addVars(data_loader_a2.F,data_loader_a2.F,data_loader_a2.C,vtype=gp.GRB.BINARY,name='w_ijk')
u_ijk=m.addVars(data_loader_a2.F,data_loader_a2.F,data_loader_a2.C,vtype=gp.GRB.BINARY,name='u_ijk')

# a1=m.addVar(1,vtype=gp.GRB.INTEGER,name='a1')
# a2=m.addVar(2,vtype=gp.GRB.INTEGER,name='a2')


m.ModelSense=GRB.MINIMIZE

m.setObjective(-z.sum())

#m.setObjective(x_ikdh.sum())
# m.setObjective(gp.quicksum(x_ikfo[i,k] for i in F for k in C2))
# m.setObjective(gp.quicksum(gp.quicksum(data_loader_a.DCost[k]*(gp.quicksum(d_jkfin[i,k]*data_loader_a.arrivetime[i] for i in data_loader_a.FD[t]) 
#                                                                - gp.quicksum(d_iksta[i,k]*data_loader_a.leavetime[i] for i in data_loader_a.FD[t]))/60
#                                                            for k in data_loader_a.C) for t in data_loader_a.Dates ))
# m.setObjective(a1-a2)

#m.setObjectiveN(-z.sum(),index = 0,weight =0.8)
#m.setObjectiveN(x_ikdh.sum(),index=1,weight=0.2)
#m.setObjectiveN(gp.quicksum(x_ikfo[i,k] for i in data_loader_a.F for k in data_loader_a.C2),index=2,weight=0.05)


M=10000
#对X的约束
m.addConstrs(x_ikfo[i,k]==0 for i in data_loader_a2.F for k in data_loader_a2.C1 )
m.addConstrs(x_ikcap[i,k]==0 for i in data_loader_a2.F for k in data_loader_a2.C3 )
m.addConstrs(x_ikcap[i,k]+x_ikfo[i,k]+x_ikdh[i,k]<=1 for i in data_loader_a2.F for k in data_loader_a2.C)

#对Z的约束
m.addConstrs(gp.quicksum(x_ikcap[i,k]+x_ikfo[i,k]+x_ikdh[i,k] for k in data_loader_a2.C)<=M*(z[i]) for i in data_loader_a2.F)
m.addConstrs(x_ikcap.sum(i,'*')== z[i] for i in data_loader_a2.F )
m.addConstrs(x_ikfo.sum(i,'*')== z[i] for i in data_loader_a2.F )
m.addConstrs(M*z[j]>=gp.quicksum(x_ikcap[j,k]+x_ikfo[j,k]+x_ikdh[j,k] for k in data_loader_a2.C) for j in data_loader_a2.F)

#对Y的约束
m.addConstrs(y_ijk[i,j,k]==0 for i,j in data_loader_a2.FF for k in data_loader_a2.C)
#m.addConstrs(gp.quicksum(y_ijk[i,j,k] for j in F)<=x_ikcap[i,k]+x_ikfo[i,k]+x_ikdh[i,k] for i in F for k in C)
#m.addConstrs(gp.quicksum(y_ijk[j,i,k] for j in F)<=x_ikcap[i,k]+x_ikfo[i,k]+x_ikdh[i,k] for i in F for k in C)

#对roaster周期的约束
# m.addConstrs(r_jkfin[j,k]== 0 for j in data_loader_a.nonF_arrive_base[0] for k in data_loader_a.C)
m.addConstrs(r_iksta[i,k]== 0 for i in temp_leave_dic[k] for k in data_loader_a2.C)

m.addConstrs(gp.quicksum(r_iksta[i,k] for i in data_loader_a2.F_leave_base[0] )<=1  for k in data_loader_a2.C)
m.addConstrs(gp.quicksum(r_jkfin[j,k] for j in data_loader_a2.F_arrive_base[0] )-gp.quicksum(r_iksta[i,k] for i in data_loader_a2.F_leave_base[0] ) == 0 for k in data_loader_a2.C)

#第一问中航班对应周期的约束
m.addConstrs(1-(y_ijk.sum(i,'*',k)+r_iksta[i,k]+r_jkfin[i,k])<=M*(1-(x_ikcap[i,k]+x_ikfo[i,k]+x_ikdh[i,k])) for i in data_loader_a2.F for k in data_loader_a2.C)
m.addConstrs(y_ijk.sum(i,'*',k)+r_jkfin[i,k]-(y_ijk.sum('*',i,k)+r_iksta[i,k]) == 0 for i in data_loader_a2.F for k in data_loader_a2.C)

m.addConstrs(y_ijk.sum(i,'*',k)+r_jkfin[i,k] <= x_ikcap[i,k]+x_ikfo[i,k]+x_ikdh[i,k]  for i in data_loader_a2.F for k in data_loader_a2.C)
m.addConstrs(y_ijk.sum('*',i,k)+r_iksta[i,k] <= x_ikcap[i,k]+x_ikfo[i,k]+x_ikdh[i,k]  for i in data_loader_a2.F for k in data_loader_a2.C)

#第二问
#对V的约束
m.addConstrs(v_ijk[i,j,k]==0 for i,j in data_loader_a2.FF1 for k in data_loader_a2.C)
m.addConstrs(v_ijk[i,j,k] <= y_ijk[i,j,k] for i in data_loader_a2.F for j in data_loader_a.F for k in data_loader_a2.C)


#对duty执勤的约束
m.addConstrs(gp.quicksum(d_iksta[i,k] for i in data_loader_a2.FD[t]) - gp.quicksum(d_jkfin[i,k] for i in data_loader_a2.FD[t]) == 0 for i in data_loader_a2.F for k in data_loader_a2.C for t in data_loader_a2.Dates)
m.addConstrs(gp.quicksum(d_iksta[i,k] for i in data_loader_a2.FD[t])   <= gp.quicksum(x_ikcap[i,k]+x_ikfo[i,k]+x_ikdh[i,k] for i in data_loader_a2.FD[t]) for i in data_loader_a2.F for k in data_loader_a2.C for t in data_loader_a2.Dates)
m.addConstrs(gp.quicksum(d_iksta[i,k] for i in data_loader_a2.FD[t])*M >= gp.quicksum(x_ikcap[i,k]+x_ikfo[i,k]+x_ikdh[i,k] for i in data_loader_a2.FD[t]) for k in data_loader_a2.C for t in data_loader_a2.Dates)

#m.addConstrs(d_iksta[i,k] for i in data_loader_a.FD[t]  <= x_ikcap[i,k]+x_ikfo[i,k]+x_ikdh[i,k] for i in data_loader_a.FD[t] for k in data_loader_a.C for t in data_loader_a.Dates)

#第二问中航班对应执勤，执勤对应周期的约束
m.addConstrs((v_ijk.sum(i,'*',k)+ r_jkfin[i,k] ==  d_jkfin[i,k]) for i in data_loader_a2.F for k in data_loader_a2.C)
m.addConstrs((v_ijk.sum('*',i,k)+ r_iksta[i,k] ==  d_iksta[i,k]) for i in data_loader_a2.F for k in data_loader_a2.C)

#m.addConstrs((d_jkfin[i,k] <= x_ikcap[i,k]+x_ikfo[i,k]+x_ikdh[i,k])  for i in data_loader_a.F for k in data_loader_a.C)
#m.addConstrs((d_iksta[i,k] <= x_ikcap[i,k]+x_ikfo[i,k]+x_ikdh[i,k])  for i in data_loader_a.F for k in data_loader_a.C)

#第二问中其余约束
m.addConstrs(gp.quicksum(d_jkfin[i,k]*data_loader_a2.arrivetime[i] for i in data_loader_a2.FD[t]) - gp.quicksum(d_iksta[i,k]*data_loader_a2.leavetime[i] for i in data_loader_a2.FD[t]) <= 720 for k in data_loader_a2.C for t in data_loader_a2.Dates )
m.addConstrs(gp.quicksum((x_ikcap[i,k]+x_ikfo[i,k])*(data_loader_a2.leavetime[i]-data_loader_a2.arrivetime[i]) for i in data_loader_a2.FD[t]) <= 600 for k in data_loader_a2.C for t in data_loader_a2.Dates )
#m.addConstrs(gp.quicksum(d_jkfin[i,k]*data_loader_a.arrivetime[i] for i in data_loader_a.FD[t]) - gp.quicksum(d_iksta[i,k]*data_loader_a.leavetime[i] for i in data_loader_a.FD[t]) >= 0 for k in data_loader_a.C for t in data_loader_a.Dates )


#m.addConstr(z.sum('*')==206)
# m.addConstr(gp.quicksum(gp.quicksum(data_loader_a.DCost[k]*(gp.quicksum(d_jkfin[i,k]*data_loader_a.arrivetime[i] for i in data_loader_a.FD[t]) 
#                     - gp.quicksum(d_iksta[i,k]*data_loader_a.leavetime[i] for i in data_loader_a.FD[t]))
#                 for k in data_loader_a.C) for t in data_loader_a.Dates )==[1324573-100000,1324573+100000])
#对执勤时长平衡的约束（辅助目标）
# m.addConstrs(a1>=(gp.quicksum(gp.quicksum(d_jkfin[i,k]*data_loader_a.arrivetime[i] for i in data_loader_a.FD[t]) for t in data_loader_a.Dates) - gp.quicksum(gp.quicksum(d_iksta[i,k]*data_loader_a.leavetime[i] for i in data_loader_a.FD[t]) for t in data_loader_a.Dates)) for k in data_loader_a.C)
# m.addConstrs(a2<=(gp.quicksum(gp.quicksum(d_jkfin[i,k]*data_loader_a.arrivetime[i] for i in data_loader_a.FD[t]) for t in data_loader_a.Dates) - gp.quicksum(gp.quicksum(d_iksta[i,k]*data_loader_a.leavetime[i] for i in data_loader_a.FD[t]) for t in data_loader_a.Dates)) for k in data_loader_a.C)
         
#第三问
#对W的约束
m.addConstrs(w_ijk[i,j,k]==0 for i,j in data_loader_a2.FF2 for k in data_loader_a2.C)
m.addConstrs(w_ijk[i,j,k] <= v_ijk[i,j,k] for i in data_loader_a2.F for j in data_loader_a2.F for k in data_loader_a2.C)



#对U的约束
m.addConstrs(r_jkfin[i,k] <=gp.quicksum(u_ijk[j,i,k] for j in data_loader_a2.F) for i in data_loader_a2.F for k in data_loader_a2.C)
m.addConstrs(r_iksta[i,k] <=gp.quicksum(u_ijk[i,j,k] for j in data_loader_a2.F) for i in data_loader_a2.F for k in data_loader_a2.C)

m.addConstrs(gp.quicksum(u_ijk[j,i,k] for j in data_loader_a2.F) <= d_jkfin[i,k] for i in data_loader_a2.F for k in data_loader_a2.C)
m.addConstrs(gp.quicksum(u_ijk[i,j,k] for j in data_loader_a2.F) <= d_iksta[i,k] for i in data_loader_a2.F for k in data_loader_a2.C)

m.addConstrs(gp.quicksum(u_ijk[i,j,k] for j in data_loader_a2.F)-(r_iksta[i,k] + gp.quicksum(w_ijk[j,i,k] for j in data_loader_a2.F)) == 0 for i in data_loader_a2.F for k in data_loader_a2.C)
m.addConstrs(gp.quicksum(u_ijk[j,i,k] for j in data_loader_a2.F)-(r_jkfin[i,k] + gp.quicksum(w_ijk[i,j,k] for j in data_loader_a2.F))== 0 for i in data_loader_a2.F for k in data_loader_a2.C)


m.addConstrs(u_ijk[i,j,k]*(data_loader_a2.leavedate[j] - data_loader_a2.leavedate[i] )<= 4 for i in data_loader_a2.F for j in data_loader_a2.F for k in data_loader_a2.C)
#m.addConstrs(w_ijk[i,j,k]*(data_loader_a.leavedate[j] - data_loader_a.leavedate[i] )>= 2 for i in data_loader_a.F for j in data_loader_a.F for k in data_loader_a.C)
m.addConstrs(gp.quicksum(u_ijk[i,j,k]*(data_loader_a2.arrivetime[j] - data_loader_a2.leavetime[i])for i in data_loader_a2.F for j in data_loader_a2.F )<= MaxTAFB for k in data_loader_a2.C)       
m.addConstrs(u_ijk[i,j,k]==0 for i,j in data_loader_a2.FF for k in data_loader_a2.C)

# m.addConstr(z.sum()==58)

m.update()
m.optimize()

In [ ]:
for k in data_loader_a2.C:
    sum1 = 0
    for i in data_loader_a2.F:
        for j in data_loader_a2.F:
            if u_ijk[i,j,k].x > 0.9:
                sum1 += data_loader_a2.arrivetime[j] - data_loader_a2.leavetime[i]
    print(str(k)+": "+str(sum1))

In [ ]:
loop_dic={'A0003': [2, 1],
 'A0004': [2, 1],
 'A0001': [1, 2],
 'A0013': [2, 1],
 'A0021': [4],
 'A0015': [4],
 'A0008': [4],
 'A0019': [2, 1],
 'A0002': [1, 2],
 'A0009': [4],
 'A0020': [4],
 'A0010': [4],
 'A0005': [4],
 'A0006': [4],
 'A0018': [4],
 'A0012': [1, 2],
 'A0011': [4],
 'A0017': [4],
 'A0007': [4],
 'A0014': [4],
 'A0016': [4]}

In [ ]:
for i in data_loader_a2.F:
    for j in data_loader_a2.F:
        for k in data_loader_a2.C:
            if u_ijk[i,j,k].x>0.9:
                if data_loader_a2.C_dic[k] in loop_dic.keys():
                    loop_dic[data_loader_a2.C_dic[k]].append(data_loader_a2.leavedate[j]-data_loader_a2.leavedate[i]+1)
                else:
                    loop_dic[data_loader_a2.C_dic[k]]=[data_loader_a2.leavedate[j]-data_loader_a2.leavedate[i]+1]
loop_dic

In [ ]:
sum = 0
for i in data_loader_a2.F:
    for j in data_loader_a2.F:
        for k in data_loader_a2.C:
                sum+=data_loader_a2.PCost[k]*(u_ijk[i,j,k].x*data_loader_a2.arrivetime[j] - u_ijk[i,j,k].x*data_loader_a2.leavetime[i])/60
sum

In [ ]:
for i in data_loader_a2.F:
    for k in data_loader_a2.C:
        if x_ikcap[i,k].x > 0.9:
            results_dic['em_no'].append(k)
            for key, value in data_loader.F_dic.items():
                if value == data_loader_a2.F_dic[i]:
                    results_dic['fl_no'].append(key)
                    break
            results_dic['cls'].append(1)
        if x_ikfo[i,k].x > 0.9:
            results_dic['em_no'].append(k)
            for key, value in data_loader.F_dic.items():
                if value == data_loader_a2.F_dic[i]:
                    results_dic['fl_no'].append(key)
                    break
            results_dic['cls'].append(2)
        if x_ikdh[i,k].x > 0.9:
            results_dic['em_no'].append(k)
            for key, value in data_loader.F_dic.items():
                if value == data_loader_a2.F_dic[i]:
                    results_dic['fl_no'].append(key)
                    break
            results_dic['cls'].append(3)

In [ ]:
data_loader_a3 = DataLoader('data/Data A-Crew.csv', 'data/Data A-Flight.csv')
data_loader_a3.dump_data(cropped_date=(21, 25))

In [ ]:
for i in data_loader_a3.F:
    for k in data_loader_a3.C:
        if r_iksta[i,k].x > 0.9:
            print(str(data_loader_a3.C_dic[k])+"乘坐"+str(data_loader_a3.F_dic[i]))

In [ ]:
data_loader_a3.C

In [ ]:
temp_leave = {}
for i in data_loader_a3.F:
    for k in data_loader_a3.C:
        if r_jkfin[i,k].x > 0.9:
            print(str(data_loader_a3.C_dic[k])+"乘坐"+str(data_loader_a3.F_dic[i]))
            temp_leave[k] = i
temp_leave

In [ ]:
data_loader_a3.AP_dic

In [ ]:
data_loader_a3.F_ap_dpt_dic

In [ ]:
data_loader_a3.leavedate

In [ ]:
data_loader_a2.leavedate

In [ ]:
temp_leave

In [ ]:
temp_leave_dic = {}
MinRest = 2
for k in data_loader_a3.C:
    for i in data_loader_a3.F:
        if k not in temp_leave.keys():
            temp_leave_dic[k] = []
            continue
        if ((data_loader_a3.AP_dic[data_loader_a3.F_ap_dpt_dic[i]] == 'NKX') and ((data_loader_a3.leavedate[i] - data_loader_a2.leavedate[temp_leave[k]])>MinRest)):#不是到达的机场
            continue
        else:
            if k not in temp_leave_dic.keys(): 
                temp_leave_dic[k] = [i]
            else:
                temp_leave_dic[k].append(i)

## 第三阶段

### 优化目标①

In [ ]:
MaxTAFB = 14400 / 15 * 5

m=gp.Model('m1')

z=m.addVars(data_loader_a3.F,vtype=gp.GRB.BINARY,name='z')
x_ikdh=m.addVars(data_loader_a3.F,data_loader_a3.C,vtype=gp.GRB.BINARY,name='x_dh')
x_ikfo=m.addVars(data_loader_a3.F,data_loader_a3.C,vtype=gp.GRB.BINARY,name='x_fo')
x_ikcap=m.addVars(data_loader_a3.F,data_loader_a3.C,vtype=gp.GRB.BINARY,name='x_cap')

r_iksta=m.addVars(data_loader_a3.F,data_loader_a3.C,vtype=gp.GRB.BINARY,name='r_iksta')
r_jkfin=m.addVars(data_loader_a3.F,data_loader_a3.C,vtype=gp.GRB.BINARY,name='r_jkfin')
d_iksta=m.addVars(data_loader_a3.F,data_loader_a3.C,vtype=gp.GRB.BINARY,name='d_iksta')
d_jkfin=m.addVars(data_loader_a3.F,data_loader_a3.C,vtype=gp.GRB.BINARY,name='d_jkfin')
p_iksta=m.addVars(data_loader_a3.F,data_loader_a3.C,vtype=gp.GRB.BINARY,name='p_iksta')
p_jkfin=m.addVars(data_loader_a3.F,data_loader_a3.C,vtype=gp.GRB.BINARY,name='p_jkfin')

y_ijk=m.addVars(data_loader_a3.F,data_loader_a3.F,data_loader_a3.C,vtype=gp.GRB.BINARY,name='y_ijk')
v_ijk=m.addVars(data_loader_a3.F,data_loader_a3.F,data_loader_a3.C,vtype=gp.GRB.BINARY,name='v_ijk')
w_ijk=m.addVars(data_loader_a3.F,data_loader_a3.F,data_loader_a3.C,vtype=gp.GRB.BINARY,name='w_ijk')
u_ijk=m.addVars(data_loader_a3.F,data_loader_a3.F,data_loader_a3.C,vtype=gp.GRB.BINARY,name='u_ijk')

# a1=m.addVar(1,vtype=gp.GRB.INTEGER,name='a1')
# a2=m.addVar(2,vtype=gp.GRB.INTEGER,name='a2')


m.ModelSense=GRB.MINIMIZE

m.setObjective(-z.sum())

#m.setObjective(x_ikdh.sum())
# m.setObjective(gp.quicksum(x_ikfo[i,k] for i in F for k in C2))
# m.setObjective(gp.quicksum(gp.quicksum(data_loader_a.DCost[k]*(gp.quicksum(d_jkfin[i,k]*data_loader_a.arrivetime[i] for i in data_loader_a.FD[t]) 
#                                                                - gp.quicksum(d_iksta[i,k]*data_loader_a.leavetime[i] for i in data_loader_a.FD[t]))
#                                                            for k in data_loader_a.C) for t in data_loader_a.Dates ))
# m.setObjective(a1-a2)

#m.setObjectiveN(-z.sum(),index = 0,weight =0.8)
#m.setObjectiveN(x_ikdh.sum(),index=1,weight=0.2)
#m.setObjectiveN(gp.quicksum(x_ikfo[i,k] for i in data_loader_a.F for k in data_loader_a.C2),index=2,weight=0.05)


M=10000
#对X的约束
m.addConstrs(x_ikfo[i,k]==0 for i in data_loader_a3.F for k in data_loader_a3.C1 )
m.addConstrs(x_ikcap[i,k]==0 for i in data_loader_a3.F for k in data_loader_a3.C3 )
m.addConstrs(x_ikcap[i,k]+x_ikfo[i,k]+x_ikdh[i,k]<=1 for i in data_loader_a3.F for k in data_loader_a3.C)

#对Z的约束
m.addConstrs(gp.quicksum(x_ikcap[i,k]+x_ikfo[i,k]+x_ikdh[i,k] for k in data_loader_a3.C)<=M*(z[i]) for i in data_loader_a3.F)
m.addConstrs(x_ikcap.sum(i,'*')== z[i] for i in data_loader_a3.F )
m.addConstrs(x_ikfo.sum(i,'*')== z[i] for i in data_loader_a3.F )
m.addConstrs(M*z[j]>=gp.quicksum(x_ikcap[j,k]+x_ikfo[j,k]+x_ikdh[j,k] for k in data_loader_a3.C) for j in data_loader_a3.F)

#对Y的约束
m.addConstrs(y_ijk[i,j,k]==0 for i,j in data_loader_a3.FF for k in data_loader_a3.C)
#m.addConstrs(gp.quicksum(y_ijk[i,j,k] for j in F)<=x_ikcap[i,k]+x_ikfo[i,k]+x_ikdh[i,k] for i in F for k in C)
#m.addConstrs(gp.quicksum(y_ijk[j,i,k] for j in F)<=x_ikcap[i,k]+x_ikfo[i,k]+x_ikdh[i,k] for i in F for k in C)

#对roaster周期的约束
# m.addConstrs(r_jkfin[j,k]== 0 for j in data_loader_a.nonF_arrive_base[0] for k in data_loader_a.C)
m.addConstrs(r_iksta[i,k]== 0 for i in temp_leave_dic[k] for k in data_loader_a3.C)

m.addConstrs(gp.quicksum(r_iksta[i,k] for i in data_loader_a3.F_leave_base[0] )<=1  for k in data_loader_a3.C)
m.addConstrs(gp.quicksum(r_jkfin[j,k] for j in data_loader_a3.F_arrive_base[0] )-gp.quicksum(r_iksta[i,k] for i in data_loader_a3.F_leave_base[0] ) == 0 for k in data_loader_a3.C)

#第一问中航班对应周期的约束
m.addConstrs(1-(y_ijk.sum(i,'*',k)+r_iksta[i,k]+r_jkfin[i,k])<=M*(1-(x_ikcap[i,k]+x_ikfo[i,k]+x_ikdh[i,k])) for i in data_loader_a3.F for k in data_loader_a3.C)
m.addConstrs(y_ijk.sum(i,'*',k)+r_jkfin[i,k]-(y_ijk.sum('*',i,k)+r_iksta[i,k]) == 0 for i in data_loader_a3.F for k in data_loader_a3.C)

m.addConstrs(y_ijk.sum(i,'*',k)+r_jkfin[i,k] <= x_ikcap[i,k]+x_ikfo[i,k]+x_ikdh[i,k]  for i in data_loader_a3.F for k in data_loader_a3.C)
m.addConstrs(y_ijk.sum('*',i,k)+r_iksta[i,k] <= x_ikcap[i,k]+x_ikfo[i,k]+x_ikdh[i,k]  for i in data_loader_a3.F for k in data_loader_a3.C)

#第二问
#对V的约束
m.addConstrs(v_ijk[i,j,k]==0 for i,j in data_loader_a3.FF1 for k in data_loader_a3.C)
m.addConstrs(v_ijk[i,j,k] <= y_ijk[i,j,k] for i in data_loader_a3.F for j in data_loader_a.F for k in data_loader_a3.C)


#对duty执勤的约束
m.addConstrs(gp.quicksum(d_iksta[i,k] for i in data_loader_a3.FD[t]) - gp.quicksum(d_jkfin[i,k] for i in data_loader_a3.FD[t]) == 0 for i in data_loader_a3.F for k in data_loader_a3.C for t in data_loader_a3.Dates)
m.addConstrs(gp.quicksum(d_iksta[i,k] for i in data_loader_a3.FD[t])   <= gp.quicksum(x_ikcap[i,k]+x_ikfo[i,k]+x_ikdh[i,k] for i in data_loader_a3.FD[t]) for i in data_loader_a3.F for k in data_loader_a3.C for t in data_loader_a3.Dates)
m.addConstrs(gp.quicksum(d_iksta[i,k] for i in data_loader_a3.FD[t])*M >= gp.quicksum(x_ikcap[i,k]+x_ikfo[i,k]+x_ikdh[i,k] for i in data_loader_a3.FD[t]) for k in data_loader_a3.C for t in data_loader_a3.Dates)

#m.addConstrs(d_iksta[i,k] for i in data_loader_a.FD[t]  <= x_ikcap[i,k]+x_ikfo[i,k]+x_ikdh[i,k] for i in data_loader_a.FD[t] for k in data_loader_a.C for t in data_loader_a.Dates)

#第二问中航班对应执勤，执勤对应周期的约束
m.addConstrs((v_ijk.sum(i,'*',k)+ r_jkfin[i,k] ==  d_jkfin[i,k]) for i in data_loader_a3.F for k in data_loader_a3.C)
m.addConstrs((v_ijk.sum('*',i,k)+ r_iksta[i,k] ==  d_iksta[i,k]) for i in data_loader_a3.F for k in data_loader_a3.C)

#m.addConstrs((d_jkfin[i,k] <= x_ikcap[i,k]+x_ikfo[i,k]+x_ikdh[i,k])  for i in data_loader_a.F for k in data_loader_a.C)
#m.addConstrs((d_iksta[i,k] <= x_ikcap[i,k]+x_ikfo[i,k]+x_ikdh[i,k])  for i in data_loader_a.F for k in data_loader_a.C)

#第二问中其余约束
m.addConstrs(gp.quicksum(d_jkfin[i,k]*data_loader_a3.arrivetime[i] for i in data_loader_a3.FD[t]) - gp.quicksum(d_iksta[i,k]*data_loader_a3.leavetime[i] for i in data_loader_a3.FD[t]) <= 720 for k in data_loader_a3.C for t in data_loader_a3.Dates )
m.addConstrs(gp.quicksum((x_ikcap[i,k]+x_ikfo[i,k])*(data_loader_a3.leavetime[i]-data_loader_a3.arrivetime[i]) for i in data_loader_a3.FD[t]) <= 600 for k in data_loader_a3.C for t in data_loader_a3.Dates )
#m.addConstrs(gp.quicksum(d_jkfin[i,k]*data_loader_a.arrivetime[i] for i in data_loader_a.FD[t]) - gp.quicksum(d_iksta[i,k]*data_loader_a.leavetime[i] for i in data_loader_a.FD[t]) >= 0 for k in data_loader_a.C for t in data_loader_a.Dates )


#m.addConstr(z.sum('*')==206)
# m.addConstr(gp.quicksum(gp.quicksum(data_loader_a.DCost[k]*(gp.quicksum(d_jkfin[i,k]*data_loader_a.arrivetime[i] for i in data_loader_a.FD[t]) 
#                     - gp.quicksum(d_iksta[i,k]*data_loader_a.leavetime[i] for i in data_loader_a.FD[t]))
#                 for k in data_loader_a.C) for t in data_loader_a.Dates )==[1324573-100000,1324573+100000])
#对执勤时长平衡的约束（辅助目标）
# m.addConstrs(a1>=(gp.quicksum(gp.quicksum(d_jkfin[i,k]*data_loader_a.arrivetime[i] for i in data_loader_a.FD[t]) for t in data_loader_a.Dates) - gp.quicksum(gp.quicksum(d_iksta[i,k]*data_loader_a.leavetime[i] for i in data_loader_a.FD[t]) for t in data_loader_a.Dates)) for k in data_loader_a.C)
# m.addConstrs(a2<=(gp.quicksum(gp.quicksum(d_jkfin[i,k]*data_loader_a.arrivetime[i] for i in data_loader_a.FD[t]) for t in data_loader_a.Dates) - gp.quicksum(gp.quicksum(d_iksta[i,k]*data_loader_a.leavetime[i] for i in data_loader_a.FD[t]) for t in data_loader_a.Dates)) for k in data_loader_a.C)
         
#第三问
#对W的约束
m.addConstrs(w_ijk[i,j,k]==0 for i,j in data_loader_a3.FF2 for k in data_loader_a3.C)
m.addConstrs(w_ijk[i,j,k] <= v_ijk[i,j,k] for i in data_loader_a3.F for j in data_loader_a3.F for k in data_loader_a3.C)



#对U的约束
m.addConstrs(r_jkfin[i,k] <=gp.quicksum(u_ijk[j,i,k] for j in data_loader_a3.F) for i in data_loader_a3.F for k in data_loader_a3.C)
m.addConstrs(r_iksta[i,k] <=gp.quicksum(u_ijk[i,j,k] for j in data_loader_a3.F) for i in data_loader_a3.F for k in data_loader_a3.C)

m.addConstrs(gp.quicksum(u_ijk[j,i,k] for j in data_loader_a3.F) <= d_jkfin[i,k] for i in data_loader_a3.F for k in data_loader_a3.C)
m.addConstrs(gp.quicksum(u_ijk[i,j,k] for j in data_loader_a3.F) <= d_iksta[i,k] for i in data_loader_a3.F for k in data_loader_a3.C)

m.addConstrs(gp.quicksum(u_ijk[i,j,k] for j in data_loader_a3.F)-(r_iksta[i,k] + gp.quicksum(w_ijk[j,i,k] for j in data_loader_a3.F)) == 0 for i in data_loader_a3.F for k in data_loader_a3.C)
m.addConstrs(gp.quicksum(u_ijk[j,i,k] for j in data_loader_a3.F)-(r_jkfin[i,k] + gp.quicksum(w_ijk[i,j,k] for j in data_loader_a3.F))== 0 for i in data_loader_a3.F for k in data_loader_a3.C)


m.addConstrs(u_ijk[i,j,k]*(data_loader_a3.leavedate[j] - data_loader_a3.leavedate[i] )<= 4 for i in data_loader_a3.F for j in data_loader_a3.F for k in data_loader_a3.C)
#m.addConstrs(w_ijk[i,j,k]*(data_loader_a.leavedate[j] - data_loader_a.leavedate[i] )>= 2 for i in data_loader_a.F for j in data_loader_a.F for k in data_loader_a.C)
m.addConstrs(gp.quicksum(u_ijk[i,j,k]*(data_loader_a3.arrivetime[j] - data_loader_a3.leavetime[i])for i in data_loader_a3.F for j in data_loader_a3.F )<= MaxTAFB for k in data_loader_a3.C)       
m.addConstrs(u_ijk[i,j,k]==0 for i,j in data_loader_a3.FF for k in data_loader_a3.C)

m.update()
m.optimize()

# 结果计算与导出

In [ ]:
for k in data_loader_a3.C:
    sum1 = 0
    for i in data_loader_a3.F:
        for j in data_loader_a3.F:
            if u_ijk[i,j,k].x > 0.9:
                sum1 += data_loader_a3.arrivetime[j] - data_loader_a3.leavetime[i]

In [ ]:
for i in data_loader_a3.F:
    for j in data_loader_a3.F:
        for k in data_loader_a3.C:
            if u_ijk[i,j,k].x>0.9:
                if data_loader_a3.C_dic[k] in loop_dic.keys():
                    loop_dic[data_loader_a3.C_dic[k]].append(data_loader_a3.leavedate[j]-data_loader_a3.leavedate[i]+1)
                else:
                    loop_dic[data_loader_a3.C_dic[k]] = [data_loader_a3.leavedate[j]-data_loader_a3.leavedate[i]+1]

In [ ]:
sum = 0
for i in data_loader_a3.F:
    for j in data_loader_a3.F:
        for k in data_loader_a3.C:
            sum+=data_loader_a3.PCost[k]*(u_ijk[i,j,k].x*data_loader_a3.arrivetime[j] - u_ijk[i,j,k].x*data_loader_a3.leavetime[i])/60

In [ ]:
for i in data_loader_a3.F:
    for k in data_loader_a3.C:
        if x_ikcap[i,k].x > 0.9:
            results_dic['em_no'].append(k)
            for key, value in data_loader.F_dic.items():
                if value == data_loader_a3.F_dic[i]:
                    results_dic['fl_no'].append(key)
                    break
            results_dic['cls'].append(1)
        if x_ikfo[i,k].x > 0.9:
            results_dic['em_no'].append(k)
            for key, value in data_loader.F_dic.items():
                if value == data_loader_a3.F_dic[i]:
                    results_dic['fl_no'].append(key)
                    break
            results_dic['cls'].append(2)
        if x_ikdh[i,k].x > 0.9:
            results_dic['em_no'].append(k)
            for key, value in data_loader.F_dic.items():
                if value == data_loader_a3.F_dic[i]:
                    results_dic['fl_no'].append(key)
                    break
            results_dic['cls'].append(3)

In [ ]:
counter = {1:0,2:0,3:0,4:0}
for i in range(5): 
    for ls in loop_dic.values():
        for t in ls:
            counter[t] += 1

In [ ]:
from ResultViewer import RViewer
import pandas as pd

In [ ]:
df = pd.DataFrame(results_dic)
df

In [ ]:
rv = RViewer(data_loader, data_cls='a')
rv.load_results_from_df(df)

In [ ]:
df_a = rv.get_results_df_a()
df_a

In [ ]:
df_a.to_csv('results/q3_a_UnconveredFlights.csv', index=0)

In [ ]:
df_b = rv.get_results_df_b()
df_b

In [ ]:
df_b.to_csv('results/q3_a_CrewRosters.csv', index=0)

In [ ]:
rv.draw_ef_gantt(save='results/q3_a.png')